In [ ]:
from chromatic import *
import emcee
import corner
import speclite as speclite
import glob as glob
from speclite import filters
from tqdm import tqdm
from matplotlib import cm
from matplotlib.artist import Artist
from scipy.optimize import minimize
from scipy.optimize import curve_fit
from PyAstronomy import pyasl
from specutils.spectra import Spectrum1D, SpectralRegion
from specutils.fitting import fit_generic_continuum

params = {'legend.fontsize': 'x-large',
          'figure.figsize': (6, 4),
         'axes.labelsize': 'x-large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'x-large',
         'ytick.labelsize':'x-large'}
plt.rcParams.update(params)
plt.style.use('tableau-colorblind10')

transmission_data = np.loadtxt('../data/transmission_comp.dat')
mol_wave = transmission_data[:,0]*(1e-3) * u.micron
mol_data = transmission_data[:,1]

In [ ]:
def normalized_1T_PHOENIX(Tspec=5000,wavelength=None,**kwargs):
        
    _model = get_phoenix_photons(temperature=float(Tspec),wavelength=wavelength,
                                 logg=4.4, metallicity=0.0)
    
    _m = _model[1]/np.nanmedian(_model[1])
    spectrum = Spectrum1D(flux=_m, spectral_axis=_model[0])
    with warnings.catch_warnings():  # Ignore warnings
        warnings.simplefilter('ignore')
        g1_fit = fit_generic_continuum(spectrum)
        continuum_fit = g1_fit(_model[0])
        
    normed_model = _m/continuum_fit
    
    return normed_model

In [ ]:
def plot_nres_spectra(data_flux=None, data_err=None,wavelength=None,
                      **kwargs):
        
    fig, [ax0,ax2] = plt.subplots(2,1,figsize=(9,6),sharex=True,gridspec_kw=dict(height_ratios=[1,0.5]))
    title_label=f'Visit {visit} Order {order}'
    fig.suptitle(title_label,fontsize=16)

    for l in range(len(ref_r.timelike['time'])):
        ax0.errorbar(ref_r.wavelength,ref_r.flux[:,l], yerr=ref_r.uncertainty[:,l], zorder=-1000,color='gray',
                     label='Unprocessed NRES Spectrum',fmt='',alpha=0.01)
    ax0.errorbar(wavelength, data_flux, yerr=data_err, zorder=-100,color='deepskyblue',
                 label='NRES time-averaged spectrum',fmt='',alpha=0.7)
    ax0.set_ylim(0.0,1.5)
    ax0.set_ylabel('Relative Flux',fontsize=18)

    # The bottom plot, this order's telluric transparency
    ax2.plot(this_orders_tellurics['x'], this_orders_tellurics['y'],label='Telluric Spectrum')
    ax2.axhline(0.995,color='red',label='Cutoff')
    ax2.set_xlabel(r'$\lambda$ ($\mu$m)',fontsize=18)
    ax2.set_ylabel(r'f$_{\rm{trans}}$')
    ax2.legend(loc='lower left')
    ax2.set_ylim(0.95,1.01)
    # ax2.set_yscale('log')
    ax2.set_xlim(lambda_center.value-0.0075,lambda_center.value+0.0075)

    plt.savefig(f'../figs/{visit}_order{order}_nres_spectrum.png',dpi=100)
    plt.show()
    plt.close()

In [ ]:
ordernumbers = np.arange(53,85)
visits = ['F21','S22']

for visit in tqdm(visits):
                
    for order in ordernumbers:
        
        _r = read_rainbow(f"../data/rainbows/{visit}_{order}_original.rainbow.npy")
        ref_r = _r.trim()
        lambda_center = np.nanmedian(_r.wavelength)
        this_orders_tellurics = bintogrid(mol_wave.value,mol_data,newx=ref_r.wavelength.value,visualize=False)
        nres_rainbow = read_rainbow(f"../data/rainbows/{visit}_{order}_clipped.rainbow.npy")
        data_wave = nres_rainbow.wavelength
        nres_avg_1dspec = nres_rainbow.get_average_spectrum()
        nres_avg_1derr = np.nanmedian(nres_rainbow.uncertainty,axis=1)/np.sqrt(len(nres_rainbow.timelike['time']))
        template_spec = normalized_1T_PHOENIX(temp=3600,wavelength=data_wave)
        rchisq = np.nansum((template_spec-nres_avg_1dspec)**2/(nres_avg_1derr)**2) / (len(data_wave.value)-1)
        if rchisq > 1.0:
            nres_avg_1derr = nres_avg_1derr * np.sqrt(rchisq)
         
        plot_nres_spectra(wavelength=data_wave,
                          data_flux=nres_avg_1dspec, data_err=nres_avg_1derr)